<a href="https://colab.research.google.com/github/mkhalil7625/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [0]:
##### Begin Working Here #####
import pandas as pd
import numpy as np

In [2]:
! wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
!unzip LoanStats_2018Q4.csv.zip

--2020-05-09 03:29:45--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 52.89.126.95, 54.201.144.36, 44.225.46.60
Connecting to resources.lendingclub.com (resources.lendingclub.com)|52.89.126.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [ <=>                ]  22.05M  1.74MB/s    in 15s     

2020-05-09 03:30:00 (1.49 MB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [23121687]

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [3]:
loans = pd.read_csv('LoanStats_2018Q4.csv', header=1, skipfooter=2)
loans.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,10000,10000,10000.0,60 months,16.14%,243.93,C,C4,Sale Consultant,2 years,RENT,90000.0,Not Verified,Dec-2018,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,079xx,NJ,8.80,0,Mar-1991,1,NaN,43.0,11,1,10582,58.5%,32,w,0.00,0.00,10661.057727,10661.06,...,80.0,1,0,25600,15470,15000,7500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6500,6500,6500.0,36 months,11.80%,215.28,B,B4,Educator,2 years,RENT,46500.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,352xx,AL,21.01,0,Aug-2011,1,61.0,NaN,24,0,6741,41.6%,30,w,4066.26,4066.26,3222.810000,3222.81,...,0.0,0,0,50845,40223,13300,34645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4500,4500,4500.0,36 months,11.31%,147.99,B,B3,Accounts Examiner III,10+ years,RENT,38500.0,Not Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,786xx,TX,4.64,0,Dec-2003,0,25.0,NaN,12,0,4472,15.3%,25,w,0.00,0.00,4549.217149,4549.22,...,0.0,0,0,53795,29137,15100,24595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,25000,25000,25000.0,60 months,14.47%,587.82,C,C2,teacher,10+ years,OWN,110000.0,Not Verified,Dec-2018,Charged Off,n,NaN,NaN,credit_card,Credit card refinancing,117xx,NY,26.43,1,Jan-1997,0,7.0,NaN,23,0,39053,45.7%,49,w,0.00,0.00,5848.050000,5848.05,...,10.0,0,0,179321,95648,62800,91424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,27.27%,920.91,E,E5,Claims Analyst,1 year,RENT,50000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,857xx,AZ,23.62,0,Dec-1997,1,NaN,NaN,10,0,1258,10.7%,16,w,25780.25,25780.25,13745.470000,13745.47,...,0.0,0,0,46300,27364,10100,34500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Convert the term column from string to integer.
#xamine term
loans['term'].head()

0     60 months
1     36 months
2     36 months
3     60 months
4     60 months
Name: term, dtype: object

In [5]:
loans['term'].value_counts()

 36 months    88179
 60 months    40233
Name: term, dtype: int64

In [8]:
#test the function
int(loans['term'][0].strip(' months'))
print(type(int(loans['term'][0].strip(' months'))))

<class 'int'>


In [0]:
#create teh function for stripping the str and to cast/convert to int
def myfunc(my_input_value):
    return int(my_input_value.strip(' months'))

In [10]:
loans['term'] = loans['term'].apply(myfunc)
loans['term'].head()


0    60
1    36
2    36
3    60
4    60
Name: term, dtype: int64

In [11]:
#Make a column named loan_status_is_great. It should contain the integer 1 if loan_status is "Current" or "Fully Paid." Else it should contain the integer 0.
#examine the data
loans['loan_status'].value_counts()

Current               88581
Fully Paid            28378
Charged Off            8016
Late (31-120 days)     2055
In Grace Period         732
Late (16-30 days)       327
Default                 323
Name: loan_status, dtype: int64

In [14]:
loans['loan_status_is_great']= np.where(np.logical_or(loans['loan_status'] == 'Current', loans['loan_status']=='Fully Paid'),1,0)
loans['loan_status_is_great'].value_counts()

1    116959
0     11453
Name: loan_status_is_great, dtype: int64

In [17]:
#test sample data
loans[['loan_status','loan_status_is_great']].sample(10)

,loan_status,loan_status_is_great
49892,Current,1
57135,Current,1
15704,Current,1
49806,Current,1
29040,Fully Paid,1
24097,Current,1
28189,Late (31-120 days),0
99599,Current,1
36335,Current,1
107873,Current,1


In [18]:
#Make last_pymnt_d_month and last_pymnt_d_year columns
loans['last_pymnt_d'].head()

0    May-2019
1    Mar-2020
2    Feb-2019
3    Oct-2019
4    Mar-2020
Name: last_pymnt_d, dtype: object

In [29]:
#convert to date/time format
loans['last_pymnt_d'] = pd.to_datetime(loans['last_pymnt_d'],infer_datetime_format=True,)
loans['last_pymnt_d'].head()

0   2019-05-01
1   2020-03-01
2   2019-02-01
3   2019-10-01
4   2020-03-01
Name: last_pymnt_d, dtype: datetime64[ns]

In [0]:
loans['last_pymnt_d_month'] = loans['last_pymnt_d'].dt.month


In [0]:
loans['last_pymnt_d_year'] = loans['last_pymnt_d'].dt.year

In [23]:
loans[['last_pymnt_d','last_pymnt_d_month','last_pymnt_d_year']].sample(10)
#the year and month columns were converted to float numbers

,last_pymnt_d,last_pymnt_d_month,last_pymnt_d_year
114920,2020-04-01,4.0,2020.0
111390,2019-10-01,10.0,2019.0
84812,2020-05-01,5.0,2020.0
56086,2020-04-01,4.0,2020.0
108840,2020-04-01,4.0,2020.0
109584,2020-05-01,5.0,2020.0
60771,2019-08-01,8.0,2019.0
32429,2020-04-01,4.0,2020.0
73111,2020-04-01,4.0,2020.0
127005,2019-07-01,7.0,2019.0


In [26]:
print(loans['last_pymnt_d'].dtype)

datetime64[ns]


In [31]:
#test to see if there are null values that converts my last code back to float
loans['last_pymnt_d_year'].isnull().sum()

159

In [0]:
loans.dropna(subset=['last_pymnt_d'], inplace=True)

In [0]:
loans['last_pymnt_d_month'] = loans['last_pymnt_d'].dt.month

In [0]:
loans['last_pymnt_d_year'] = loans['last_pymnt_d'].dt.year

In [37]:
loans[['last_pymnt_d','last_pymnt_d_month','last_pymnt_d_year']].sample(10)
#sucess

,last_pymnt_d,last_pymnt_d_month,last_pymnt_d_year
91613,2020-04-01,4,2020
62163,2020-04-01,4,2020
68355,2020-04-01,4,2020
27855,2020-04-01,4,2020
57020,2020-04-01,4,2020
36020,2020-06-01,6,2020
33303,2020-04-01,4,2020
102427,2020-04-01,4,2020
117246,2020-04-01,4,2020
68065,2020-04-01,4,2020


# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [38]:
 !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-05-09 04:34:51--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.104.173
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.104.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  87.4MB/s    in 2.2s    

2020-05-09 04:34:54 (87.4 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [39]:
 !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [40]:
 %cd instacart_2017_05_01

/content/instacart_2017_05_01


In [44]:
orders = pd.read_csv('orders.csv')
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
